In [196]:
import pandas as pd
import random

In [197]:
csv = pd.read_csv('input/delitos_2020.csv')

In [198]:
csv.head()

,id_mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_armas,barrio,comuna,latitud,longitud,victimas
0,RH-2020-0000480,2020,Enero,Miércoles,01/01/20,10,Hurto (sin violencia),NaN,NaN,Versalles,10.0,-34.626641,-58.527606,NaN
1,HD-2020-0001119,2020,Enero,Miércoles,01/01/20,22,Homicidio,Femicidio,NaN,Villa Devoto,11.0,-34.611268,-58.525386,1.0
2,RH-2020-0001140,2020,Enero,Miércoles,01/01/20,19,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
3,RH-2020-0001187,2020,Enero,Miércoles,01/01/20,17,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
4,RH-2020-0010817,2020,Enero,Miércoles,01/01/20,2,Hurto (sin violencia),NaN,NaN,Villa Devoto,11.0,-34.606789,-58.520726,NaN


In [199]:
# TABLA TIPOS
tipos_csv = csv[['tipo', 'subtipo']]
tipos_csv = tipos_csv.drop_duplicates(ignore_index=True)
tipos_csv = tipos_csv.fillna('Robo')
tipos_csv['id'] = tipos_csv.index + 1
tipos_csv

,tipo,subtipo,id
0,Hurto (sin violencia),Robo,1
1,Homicidio,Femicidio,2
2,Robo (con violencia),Robo,3
3,Lesiones,Siniestro Vial,4
4,Robo (con violencia),Con uso de moto,5
5,Hurto (sin violencia),Hurto Automotor,6
6,Robo (con violencia),Robo Automotor,7
7,Homicidio,Siniestro Vial,8
8,Homicidio,Doloso,9
9,Homicidio,Femicidio Intrafamiliar,10


In [200]:
# TABLA TIEMPOS
tiempos_csv = csv[['anio', 'mes', 'dia', 'fecha', 'franja']]
tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))
tiempos_csv = tiempos_csv.drop_duplicates(ignore_index=True)
tiempos_csv['id'] = tiempos_csv.franja + '_' + tiempos_csv.fecha
tiempos_csv

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_9275/595770229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))


,anio,mes,dia,fecha,franja,nro_dia,id
0,2020,Enero,Miércoles,01/01/20,10,1,10_01/01/20
1,2020,Enero,Miércoles,01/01/20,22,1,22_01/01/20
2,2020,Enero,Miércoles,01/01/20,19,1,19_01/01/20
3,2020,Enero,Miércoles,01/01/20,17,1,17_01/01/20
4,2020,Enero,Miércoles,01/01/20,2,1,2_01/01/20
...,...,...,...,...,...,...,...
8633,2020,Diciembre,Jueves,31/12/20,20,31,20_31/12/20
8634,2020,Diciembre,Jueves,31/12/20,5,31,5_31/12/20
8635,2020,Diciembre,Jueves,31/12/20,12,31,12_31/12/20
8636,2020,Diciembre,Jueves,31/12/20,6,31,6_31/12/20


In [201]:
# TABLA LUGARES
lugares_csv = csv[['barrio', 'comuna', 'latitud', 'longitud']]
# drop dups
lugares_csv = lugares_csv.drop_duplicates(ignore_index=True)
# add id column
lugares_csv['id'] = lugares_csv.index + 1
lugares_csv

,barrio,comuna,latitud,longitud,id
0,Versalles,10.0,-34.626641,-58.527606,1
1,Villa Devoto,11.0,-34.611268,-58.525386,2
2,Versalles,10.0,-34.625434,-58.525013,3
3,Villa Devoto,11.0,-34.606789,-58.520726,4
4,Mataderos,9.0,-34.655199,-58.514301,5
...,...,...,...,...,...
44389,Retiro,1.0,-34.596366,-58.378446,44390
44390,San Telmo,1.0,-34.619381,-58.37042,44391
44391,San Telmo,1.0,-34.619242,-58.366333,44392
44392,Boca,4.0,-34.636337,-58.361306,44393


In [202]:
# TABLA TIPO DE ARMA
armas_csv = csv[['uso_armas']]
armas_csv = armas_csv.drop_duplicates(ignore_index=True)
armas_csv['id'] = armas_csv.index + 1
armas_csv

,uso_armas,id
0,NaN,1
1,Arma cortante,2
2,Arma de fuego,3


In [203]:
# TABLA DELITOS
delitos_csv = csv

# create a victimas column with random values
delitos_csv['victimas'] = delitos_csv.index
delitos_csv['costo'] = delitos_csv.index
delitos_csv['victimas'] = delitos_csv['victimas'].apply(lambda x: random.randint(0, 5))
delitos_csv['costo'] = delitos_csv['costo'].apply(lambda x: random.randint(0, 100000))

In [204]:
# replace uso_armas for arma_id
delitos_csv = delitos_csv.merge(armas_csv, left_on=['uso_armas'], right_on=['uso_armas'])
delitos_csv = delitos_csv.rename(columns={'id': 'arma_id'})
delitos_csv = delitos_csv.drop(columns=['uso_armas'])

In [205]:
# add lugar_id to delitos_csv by finding a match in lugares_csv
delitos_csv = delitos_csv.merge(lugares_csv, left_on=['barrio', 'comuna', 'latitud', 'longitud'], right_on=['barrio', 'comuna', 'latitud', 'longitud'])
delitos_csv = delitos_csv.rename(columns={'id': 'lugar_id'})
delitos_csv = delitos_csv.drop(columns=['barrio', 'comuna', 'latitud', 'longitud'])

In [206]:
# replace fecha, franja for tiempo_id
delitos_csv = delitos_csv.merge(tiempos_csv, left_on=['anio', 'mes', 'dia', 'franja'], right_on=['anio', 'mes', 'dia', 'franja'])
delitos_csv = delitos_csv.rename(columns={'id': 'tiempo_id'})
delitos_csv = delitos_csv.drop(columns=['anio', 'mes', 'dia', 'franja', 'nro_dia'])
delitos_csv

,id_mapa,fecha_x,tipo,subtipo,victimas,costo,arma_id,lugar_id,fecha_y,tiempo_id
0,RH-2020-0000480,01/01/20,Hurto (sin violencia),NaN,2,99357,1,1,01/01/20,10_01/01/20
1,RH-2020-0000480,01/01/20,Hurto (sin violencia),NaN,2,99357,1,1,08/01/20,10_08/01/20
2,RH-2020-0000480,01/01/20,Hurto (sin violencia),NaN,2,99357,1,1,15/01/20,10_15/01/20
3,RH-2020-0000480,01/01/20,Hurto (sin violencia),NaN,2,99357,1,1,22/01/20,10_22/01/20
4,RH-2020-0000480,01/01/20,Hurto (sin violencia),NaN,2,99357,1,1,29/01/20,10_29/01/20
...,...,...,...,...,...,...,...,...,...,...
290055,HD-2020-0660548,15/12/20,Homicidio,Doloso,4,62332,1,42480,15/12/20,23_15/12/20
290056,HD-2020-0670159,21/12/20,Homicidio,Doloso,5,72903,1,43325,21/12/20,20_21/12/20
290057,HD-2020-0675454,25/12/20,Homicidio,Doloso,5,71217,1,43685,25/12/20,8_25/12/20
290058,RH-2020-0030320,14/01/20,Robo (con violencia),NaN,2,84685,2,3844,14/01/20,SD_14/01/20


In [207]:
# replace tipo for tipo_id
delitos_csv = delitos_csv.merge(tipos_csv, left_on=['tipo', 'subtipo'], right_on=['tipo', 'subtipo'])
delitos_csv = delitos_csv.rename(columns={'id': 'tipo_id'})
delitos_csv = delitos_csv.drop(columns=['tipo'])

In [208]:
delitos_csv = delitos_csv[['tiempo_id', 'lugar_id', 'arma_id', 'tipo_id', 'victimas', 'costo']]
delitos_csv

,tiempo_id,lugar_id,arma_id,tipo_id,victimas,costo
0,10_01/01/20,15,1,4,2,94861
1,10_08/01/20,15,1,4,2,94861
2,10_15/01/20,15,1,4,2,94861
3,10_22/01/20,15,1,4,2,94861
4,10_29/01/20,15,1,4,2,94861
...,...,...,...,...,...,...
59560,1_07/09/20,32143,1,11,4,77536
59561,1_14/09/20,32143,1,11,4,77536
59562,1_21/09/20,32143,1,11,4,77536
59563,1_28/09/20,32143,1,11,4,77536


In [209]:
# save csvs to files
tiempos_csv.to_csv('output/tiempos.csv', index=False)
lugares_csv.to_csv('output/lugares.csv', index=False)
armas_csv.to_csv('output/armas.csv', index=False)
delitos_csv.to_csv('output/delitos.csv', index=False)
tipos_csv.to_csv('output/tipos.csv', index=False)